In [1]:
%config InlineBackend.figure_formats = ['svg']
import quimb as qu
import quimb.tensor as qtn
import cotengra as ctg
import quf
import numpy as np
import torch
import algo_cooling as algo
import itertools
import algo_bp
from tqdm import tqdm
import gen_loop as lg
import gen_loop_tn as lg_tn
import autoray as ar


In [2]:
opt = algo.opt_(progbar=True)
import torch
to_backend = algo.backend_torch(device = "cpu", dtype = torch.float64, requires_grad=True)
to_backend_ = algo.backend_torch(device = "cpu", dtype = torch.float64, requires_grad=False)

opt = algo.opt_(progbar=False)
opt = "auto-hq"

In [3]:
# Define <peps | peps> as an easy problem for bp convergence:
Lx, Ly, bnd = (3, 3, 3)
L = Lx * Ly
peps = qtn.PEPS.rand(Lx=Lx, Ly=Ly, bond_dim=bnd, seed=12, dtype="float64")
fix = { f"I{i},{j}":(i,j) for i,j in itertools.product(range(peps.Lx), range(peps.Ly)) }

inds_ = { f"k{i},{j}":f"b{i},{j}"   for i in range(Lx) for j in range(Ly)}

mpo = quf.mpo_2d_His(Lx, Ly, L, data_type="float64")
pepo = quf.MPO_to_PEPO(mpo, Lx, Ly, cycle_peps= False)
pepo.show()


  ╱ 4  ╱ 8  ╱
 ●━━━━●━━━━●
╱┃1  ╱┃1  ╱┃11 
 ┃╱11 ┃╱11 ┃╱
 ●━━━━●━━━━●
╱┃11 ╱┃1  ╱┃1  
 ┃╱ 8 ┃╱ 4 ┃╱
 ●━━━━●━━━━●
╱    ╱    ╱    


In [4]:
mpo_ = quf.mpo_2d_His_(Lx,Ly,L, data_type="float64")
mpo_.show()
dmrg = qtn.DMRG2(mpo_, 
                 bond_dims=[2, 32, 64, 128, 256], 
                 cutoffs=1.e-16)
dmrg.solve(tol=1.e-14, verbosity=0, max_sweeps=20)
E_dmrg = dmrg.energy
E_dmrg / L


│4│8│11│11│11│11│8│4│
●─<─<──<──<──<──<─<─<
│ │ │  │  │  │  │ │ │


-0.5277030287280993

In [5]:
tn = peps.H & peps
norm = tn.contract(all, optimize=opt)
peps /= (norm**0.5)
tn = peps.H & peps
norm = tn.contract(all, optimize=opt)
print(complex(norm))

peps.apply_to_arrays(to_backend)
pepo.apply_to_arrays(to_backend)


(1.0000000000000002+0j)


In [6]:
pepsh = peps.H
pepsh.reindex_(inds_)
tn = pepsh & pepo & peps
E = tn.contract(all, optimize=opt)
E / L

tensor(0.0205, dtype=torch.float64, grad_fn=<DivBackward0>)

In [7]:
params, skeleton = qtn.pack(peps)
# params

In [37]:
H2 = qu.ham_heis(2) 
H2 = to_backend_(H2)

terms = {
    (coo_a, coo_b): H2
    for coo_a, coo_b in peps.gen_bond_coos()
}

In [38]:
%%time
E = peps.compute_local_expectation(
    terms,
    max_bond=None,cutoff=1e-12,
    normalized=True, contract_optimize=opt,
)
complex(E) / L

CPU times: user 2.95 s, sys: 5.39 ms, total: 2.96 s
Wall time: 118 ms


(-0.5122201399107655+0j)

In [10]:
peps.add_tag('KET')
pepsH = peps.conj().retag({'KET': 'BRA'})
tn = pepsH & peps


In [11]:
res = algo_bp.run_bp(tn, peps.site_tags, normalize=True, print_=True, opt=opt,
                    progbar=True, tol=1.e-9, max_iterations=800, 
                    update='parallel', damping=0.00, diis=True)

bp = res["bp"]
tn = res["tn"]
print("bp: zeroth", res["norm"], tn.exponent)


bp runs with the parallel update ~ iter = 800


max|dM|=1.56e-09 nconv: 6/9 : : 55it [00:00, 81.92it/s] 

messages normalized and projects are being cal
tn is being normalized w.r.t bp massages
bp: zeroth tensor(1.1475, dtype=torch.float64, grad_fn=<MulBackward0>) tensor(0.0598, dtype=torch.float64, grad_fn=<AddBackward0>)


In [12]:
%%time
#all edges in tn:
edges = [tuple(sorted(i)) for i in bp.edges.keys()]
max_edges = len(edges)
print("number of edges:", max_edges)



number of edges: 12
CPU times: user 63 µs, sys: 51 µs, total: 114 µs
Wall time: 144 µs


In [13]:
Energy_l = []

for coo_a, coo_b in peps.gen_bond_coos():
    print(coo_b, coo_a)
    res_cor = algo_bp.bp_info_rho([coo_a, coo_b])
    pass_rho = res_cor    
    obs_tensor = qtn.Tensor(H2.reshape(2,2,2,2), inds=res_cor["inds_rho"], tags="zz")




    Z_f = []
    obs_f = []
    norm_f = []
    loop_f = []
    number_edges_f = []
    obs_error_f = []
    norm_error_f = []
    norm_loop_f = []
    
    for i in range(0, max_edges+1):
        loops = lg.loop_gen_local(bp, circum = i, tags_cluster=[], sites=res_cor["reg_tags"], 
                                   circum_g=0, tn_flat=peps, site_tags=peps.site_tags)
        pass_rho |= {"chi":None}
        Z, norm_ = lg_tn.rho_bp_excited_loops_(tn, bp, pass_rho, loops, obs_tensor, opt=opt, draw_=False,
                                             pari_exclusive=res_cor["reg_tags"], simplify=False, progbar=False)
        Z_f.append(Z)
        norm_f.append(norm_)
    
    
            

    Energy_l.append(  sum(Z_f) / (sum(norm_f)*L) )
    print(  complex( sum(Z_f) / (sum(norm_f)*L) )  )
    
    

(0, 1) (0, 0)
(-0.00467267413015719+0j)
(1, 0) (0, 0)
(-0.006454751255328272+0j)
(0, 2) (0, 1)
(0.004717999975834671+0j)
(1, 1) (0, 1)
(-0.0005403960550631132+0j)
(1, 2) (0, 2)
(-0.007404800020236884+0j)
(1, 1) (1, 0)
(0.0011088706792445543+0j)
(2, 0) (1, 0)
(0.012778317853589844+0j)
(1, 2) (1, 1)
(0.003151620358909777+0j)
(2, 1) (1, 1)
(0.005325147568322872+0j)
(2, 2) (1, 2)
(0.0047702647799303215+0j)
(2, 1) (2, 0)
(0.007360710264150741+0j)
(2, 2) (2, 1)
(0.000327541265426301+0j)


In [14]:
sum(Energy_l)

tensor(0.0205, dtype=torch.float64, grad_fn=<AddBackward0>)

In [15]:
def cost(params, skeleton, max_edges=0, update='parallel'):

    params_normed = {}
    for k, t in params.items():
        norm = ar.do("norm", t)
        params_normed[k] = t / norm
    
    peps = qtn.unpack(params_normed, skeleton)
    # peps.balance_bonds_()
    # peps.equalize_norms_(1.0)
    
    peps.add_tag('KET')
    pepsH = peps.conj().retag({'KET': 'BRA'})
    tn = pepsH & peps

    res = algo_bp.run_bp(tn, peps.site_tags, normalize=True, print_=False, opt=opt,
                    progbar=False, tol=1.e-9, max_iterations=800, 
                    update=update, damping=0.00, diis=True)

    bp = res["bp"]
    tn = res["tn"]
    # print("bp: zeroth", res["norm"], tn.exponent)


    
    Energy_l = []
    
    for coo_a, coo_b in peps.gen_bond_coos():

        res_cor = algo_bp.bp_info_rho([coo_a, coo_b])
        pass_rho = res_cor    
        obs_tensor = qtn.Tensor(H2.reshape(2,2,2,2), inds=res_cor["inds_rho"], tags="zz")
    
    
    
        Z_f = []
        obs_f = []
        norm_f = []
        loop_f = []
        number_edges_f = []
        obs_error_f = []
        norm_error_f = []
        norm_loop_f = []
        
        for i in range(0, max_edges+1):
            loops = lg.loop_gen_local(bp, circum = i, tags_cluster=[], sites=res_cor["reg_tags"], 
                                       circum_g=0, tn_flat=peps, site_tags=peps.site_tags)
            pass_rho |= {"chi":None}
            Z, norm_ = lg_tn.rho_bp_excited_loops_(tn, bp, pass_rho, loops, obs_tensor, opt=opt, draw_=False,
                                                 pari_exclusive=res_cor["reg_tags"], simplify=False, progbar=False)
            Z_f.append(Z)
            norm_f.append(norm_)
        
        
                
    
        Energy_l.append(  sum(Z_f)/sum(norm_f) )
        #print(  complex( sum(Z_f)/sum(norm_f) )  )
        
    # print(sum(norm_f))

    
    return sum(Energy_l).real / L

def cost_(params, skeleton):
    peps = qtn.unpack(params, skeleton)
    
    # E = peps.compute_local_expectation(
    #     terms,
    #     max_bond=None,cutoff=1e-12,
    #     normalized=True, contract_optimize=opt,
    # )

    pepsh = peps.H
    pepsh.reindex_(inds_)
    tn = pepsh & pepo & peps
    E = tn.contract(all, optimize=opt)
    
    tn = peps.H & peps
    norm = tn.contract(all, optimize=opt)

    E = E / L
    return E.real / norm.real


In [16]:
cost(params, skeleton, max_edges=0, update='parallel')

tensor(0.0184, dtype=torch.float64, grad_fn=<DivBackward0>)

In [17]:
cost_(params, skeleton)

tensor(0.0205, dtype=torch.float64, grad_fn=<DivBackward0>)

In [18]:
import torch.optim as optim

optimizer = optim.Adam(params.values(), lr=0.01)
its_max = 400

loss_history = []
pbar = tqdm(total=its_max, desc="adam", ncols=100, dynamic_ncols=True)

for step in range(its_max):
    optimizer.zero_grad()


    loss = cost(params, skeleton, max_edges=1, update="parallel")
    loss.backward(retain_graph=False)  # keep graph if needed
    optimizer.step()
    # loss_ = cost(params, skeleton, max_edges=12, update="parallel") 
    loss_ = cost_(params, skeleton) 

    loss_history.append(loss.item())   # store scalar for plotting
    
    #print(f"Step {step}: Loss = {loss.item()}")
    pbar.set_postfix({
            "loss": f"{loss.item():.3e}","loss_": f"{complex(loss_).real:.3e}",
        })
    pbar.update(1)
pbar.close()




adam: 100%|███████████████████████████████████████████████████████| 400/400 [02:42<00:00,  2.47it/s, loss=-4.943e-01, loss_=-5.122e-01]


In [19]:
# import torch.optim as optim

# optimizer = optim.Adam(params.values(), lr=0.01)
# its_max = 1

# loss_history = []
# pbar = tqdm(total=its_max, desc="adam", ncols=100, dynamic_ncols=True)

# for step in range(its_max):
#     optimizer.zero_grad()
#     loss = cost_(params, skeleton)
#     loss.backward(retain_graph=True)  # keep graph if needed
#     optimizer.step()
    
#     loss_history.append(loss.item())   # store scalar for plotting
    
#     #print(f"Step {step}: Loss = {loss.item()}")
#     pbar.set_postfix({
#             "loss": f"{loss.item():.3e}",
#         })
#     pbar.update(1)
# pbar.close()



In [32]:
Lx = 3
Ly = 3

ham = qtn.LocalHam2D(Lx, Ly, H2=to_backend_(qu.ham_heis(2)), cyclic=False)
su = qtn.SimpleUpdate(
    peps,
    ham,
    chi=None,  # boundary contraction bond dim for computing energy
    compute_energy_every=10,
    compute_energy_per_site=True,
    keep_best=True,
)

In [33]:
for tau in [0.3, 0.1, 0.03, 0.01,0.005]:
    su.evolve(100, tau=tau)

n=500, D=3, tau=0.005, energy≈-0.517068: 100%|#######################################################| 100/100 [00:02<00:00, 42.52it/s]


In [41]:
%%time
peps = su.best['state']
E = peps.compute_local_expectation(
    terms,
    max_bond=None,cutoff=1e-12,
    normalized=True, contract_optimize=opt,
)
complex(E) / L

CPU times: user 2.96 s, sys: 30.4 ms, total: 2.99 s
Wall time: 116 ms


(-0.5173311129839597+0j)

In [44]:
pepsh = peps.H
pepsh.reindex_(inds_)
tn = pepsh & pepo & peps
E = tn.contract(all, optimize=opt)

tn = peps.H & peps
norm = tn.contract(all, optimize=opt)

E = E / (norm * L)
complex(E)

(-0.517331112983957+0j)